# LDA és TSNE bemutatása

## Feladat rövid leírása
A feladat során a kisméretű, newsgroups-small.json felhasználásával mutatom be az LDA és TSNE vizualizációs technikákat.

Az eredeti (nagyobb) adathalmaz megtalálható:
https://www.kaggle.com/datasets/crawford/20-newsgroups/code

## Adatok előkészítése

### Importok, installok

In [ ]:
import numpy as np
import pandas as pd
import re
!pip install gensim
!pip install spacy
!pip install bokeh
import gensim, spacy
from gensim.utils import simple_preprocess
import gensim.corpora as corpora
from pprint import pprint
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'not', 'would', 'say', 'could', 
                   '_', 'be', 'know', 'good', 'go', 'get', 'do', 'done', 'try', 'many', 
                   'some', 'nice', 'thank', 'think', 'see', 'rather', 'easy', 'easily', 
                   'lot', 'lack', 'make', 'want', 'seem', 'run', 'need', 'even', 'right', 
                   'line', 'even', 'also', 'may', 'take', 'come'])

!python -m spacy download en_core_web_sm

#LDA-hoz install, import
!pip install pyLDAvis
import pyLDAvis.gensim_models

#TSNE
from sklearn.manifold import TSNE
from bokeh.plotting import figure, output_file, show
from bokeh.models import Label
from bokeh.io import output_notebook
import matplotlib.colors as mcolors

import warnings
warnings.filterwarnings("ignore")

### Nyers adat beolvasása

In [ ]:
df = pd.read_json('../data/newsgroups-small.json')
df = df.loc[df.target_names.isin(['comp.sys.mac.hardware', 'rec.sport.hockey', 'talk.politics.guns', 'rec.autos']),:]
print(df.shape)
df.head()

### Szövegtisztítás
- emailek eltávolítása
- newline karakterek eltávolítása
- ' ' idézőjelek eltávolítása

In [ ]:
def sent_to_words(sentences):
    for sent in sentences:
        sent = re.sub('\\S*@\\S*\\s?', '', sent) 
        sent = re.sub('\\s+', ' ', sent)  
        sent = re.sub("\\'", "", sent) 
        sent = gensim.utils.simple_preprocess(str(sent), deacc=True) 
        yield(sent)  

#### Listává konvertálás, első sor kiíratása

In [ ]:
data = df.content.values.tolist()
data_words = list(sent_to_words(data))
print(data_words[:1])

### Bigram és trigam modell felépítése

In [ ]:
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100)
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

### Stopwordok eltávolítása a Bigram és Trigam modellekből, lemmatizáció

In [ ]:
def process_words(texts, stop_words=stop_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    texts = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
    texts = [bigram_mod[doc] for doc in texts]
    texts = [trigram_mod[bigram_mod[doc]] for doc in texts]
    texts_out = []
    nlp = spacy.load('en_core_web_sm')
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    # stopword eltávolítása a lemmatizáció után(a biztonság kedvéért)
    texts_out = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts_out]    
    return texts_out

data_ready = process_words(data_words)

### Szótárkészítés

In [ ]:
id2word = corpora.Dictionary(data_ready)

### Corpus létrehozása a kifejezések gyakoriságához

In [ ]:
corpus = [id2word.doc2bow(text) for text in data_ready]

# LDA 

In [ ]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=4, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=10,
                                           passes=10,
                                           alpha='symmetric',
                                           iterations=100,
                                           per_word_topics=True)

## LDA témáinak kiíratása

In [ ]:
pprint(lda_model.print_topics())

## LDA vizualizáció

In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary=lda_model.id2word)
vis

#### Témaeloszlások (topic weights)

In [ ]:
topic_weights = []
for i, row_list in enumerate(lda_model[corpus]):
    topic_weights.append([w for i, w in row_list[0]])

#### Tömb a témaeloszlásokról

In [ ]:
arr = pd.DataFrame(topic_weights).fillna(0).values

#### Jól elszeparált pontok megőrzése

In [ ]:
arr = arr[np.amax(arr, axis=1) > 0.35]

#### A kiemelkedő témák számának meghatározása

In [ ]:
topic_num = np.argmax(arr, axis=1)

## TSNE

#### Dimenzió redukció

In [ ]:
tsne_model = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=300, init='pca')
tsne_lda = tsne_model.fit_transform(arr)

#### Téma clusterek kirajzolása Bokeh-val

In [ ]:
output_notebook()
n_topics = 4
mycolors = np.array([color for name, color in mcolors.TABLEAU_COLORS.items()])
plot = figure(title="t-SNE Clustering az {} LDA Témákról".format(n_topics))
plot.scatter(x=tsne_lda[:,0], y=tsne_lda[:,1], color=mycolors[topic_num])
show(plot)